In [55]:
import numpy as np
from pathlib import Path
import functools


In [42]:
with Path("../14.in").open() as f:
    data = f.read().splitlines()
data = np.array([list(row) for row in data])


In [43]:
# "O" is a movable rock
# "#" is a stationary rock
# "." is free space

testdata = """\
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....""".splitlines()
testdata = np.array([list(row) for row in testdata])


In [57]:
direction_to_rotations = {
    "north": 0,
    "east": 3,
    "south": 2,
    "west": 1
}


def tilt_dish(dish, direction="north"):
    rotations = direction_to_rotations[direction]
    rotated_dish = np.rot90(dish, k=rotations)

    for ci in range(rotated_dish.shape[1]):
        last_free = -1
        for ri in range(rotated_dish.shape[0]):
            curr = rotated_dish[ri, ci]
            if curr == ".":
                if last_free == -1:
                    last_free = ri
            elif curr == "#":
                last_free = -1
            elif curr == "O":
                if last_free != -1:
                    rotated_dish[last_free, ci] = "O"
                    rotated_dish[ri, ci] = "."
                    last_free += 1

    modified_dish = np.rot90(rotated_dish, k=-rotations)
    return modified_dish


def calculate_load(dish, direction="north"):
    rotations = direction_to_rotations[direction]
    rotated_dish = np.rot90(dish, k=rotations)
    total_load = 0

    rocks_per_row = np.sum(rotated_dish == "O", axis=1)
    total_load = np.sum(rocks_per_row * np.arange(1, rotated_dish.shape[1]+1)[::-1])

    return total_load


def solve1(dish):
    modified_dish = tilt_dish(dish.copy(), direction="north")
    total_load = calculate_load(modified_dish, direction="north")
    print(f"Part 1: {total_load}")
    return total_load

assert solve1(testdata) == 136


Part 1: 136


In [45]:
solve1(data)


Part 1: 109755


109755

In [62]:
def solve2(dish, cycles=1_000_000_000):
    current_dish = dish.copy()
    cycle = ["north", "west", "south", "east"]

    cache_data = {}
    cache_idx = {}
    for i in range(cycles):
        direction = cycle[i%4]
        key = (current_dish.data.tobytes(), direction)
        modified_dish = tilt_dish(current_dish, direction=direction)
        if key in cache_data:
            j = cache_idx[key]
            print(f"Cycle found at {j} to {i}!")
            break
        cache_data[key] = modified_dish
        cache_idx[key] = i

    total_load = calculate_load(modified_dish, direction="north")
    print(f"Part 2: {total_load}")
    return total_load

solve2(testdata, cycles=1_000)
# I don't think this should be brute-forced ...
# Old Mac:  0.3 sec for 1_000
#          2:43 min for 1_000_000
#           ~45 min for 1_000_000_000 ??
# And thats just the testdata ...


Cycle found at 16 to 20!
Part 2: 115


115